In [188]:
import nltk
#nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.tokenize import sent_tokenize , word_tokenize
import re
import numpy as np
import sys
Stopwords = set(stopwords.words('english'))

##Febryan Adi Pratama - A11.2020.12719

In [189]:
def finding_all_unique_words_and_freq(words):
    words_unique = []
    word_freq = {}
    for word in words:
        if word not in words_unique:
            words_unique.append(word)
    for word in words_unique:
        word_freq[word] = words.count(word)
    return word_freq

def remove_special_characters(text):
    regex = re.compile('[^a-zA-Z0-9\s]')
    text_returned = re.sub(regex, '', text)
    return text_returned

def process_text(text):
    text = remove_special_characters(text)
    text = re.sub(re.compile('\d'), '', text)
    sentences = sent_tokenize(text)
    words = word_tokenize(text)
    words = [word for word in words if len(word) > 1]
    words = [word.lower() for word in words]
    words = [word for word in words if word not in Stopwords]
    return words

In [190]:
def main(texts):
    dict_global = {}
    idx = 1
    files_with_index = {}

    for text in texts:
        words = process_text(text)
        #print(words)
        dict_global.update(finding_all_unique_words_and_freq(words))
        files_with_index[idx] = str(idx)
        idx = idx + 1

    unique_words_all = set(dict_global.keys())

    class Node:
        def __init__(self, docId, freq=None):
            self.freq = freq
            self.doc = docId
            self.nextval = None

    class SlinkedList:
        def __init__(self, head=None):
            self.head = head

    linked_list_data = {}
    for word in unique_words_all:
        linked_list_data[word] = SlinkedList()
        linked_list_data[word].head = Node(1, Node)
    
    word_freq_in_doc = {}
    idx = 1

    for text in texts:
        words = process_text(text)
        word_freq_in_doc = finding_all_unique_words_and_freq(words)
        for word in word_freq_in_doc.keys():
            linked_list = linked_list_data[word].head
            while linked_list.nextval is not None:
                linked_list = linked_list.nextval
            linked_list.nextval = Node(idx, word_freq_in_doc[word])
        idx = idx + 1

    query = input('Enter your query:')
    query = word_tokenize(query)
    connecting_words = []
    different_words = []
    bitwise_op = []

    print(query)
    for word in query:
        if word.lower() != "and" and word.lower() != "or" and word.lower() != "not":
            different_words.append(word.lower())
        else:
            connecting_words.append(word.lower())

    total_files = len(files_with_index)
    zeroes_and_ones = []
    zeroes_and_ones_of_all_words = []

    print(different_words)
    for word in different_words:
        if word.lower() in unique_words_all:
            zeroes_and_ones = [0] * total_files
            linkedlist = linked_list_data[word].head
            while linkedlist.nextval is not None:
                zeroes_and_ones[linkedlist.nextval.doc - 1] = 1
                linkedlist = linkedlist.nextval
            zeroes_and_ones_of_all_words.append(zeroes_and_ones)
        else:
            print(word, " not found")
            sys.exit()

    print(connecting_words)

    for word in connecting_words:
        print(zeroes_and_ones_of_all_words)
        word_list1 = zeroes_and_ones_of_all_words[0]
        word_list2 = zeroes_and_ones_of_all_words[1]
        if word == "and":
            print('\n and arg')
            print(zeroes_and_ones_of_all_words)
            
            bitwise_op = [w1 & w2 for (w1, w2) in zip(word_list1, word_list2)]
            zeroes_and_ones_of_all_words.remove(word_list1)
            zeroes_and_ones_of_all_words.remove(word_list2)
            zeroes_and_ones_of_all_words.insert(0, bitwise_op)

        elif word == "or":
            print('\n or arg')
            print(zeroes_and_ones_of_all_words)

            bitwise_op = [w1 | w2 for (w1, w2) in zip(word_list1, word_list2)]
            zeroes_and_ones_of_all_words.remove(word_list1)
            zeroes_and_ones_of_all_words.remove(word_list2)
            zeroes_and_ones_of_all_words.insert(0, bitwise_op)
            print("bitwise")
            print(bitwise_op)

        elif word == "not":
            print('\n not arg')
            print(zeroes_and_ones_of_all_words)
            
            bitwise_op = [not w1 for w1 in word_list2]
            bitwise_op = [int(b == True) for b in bitwise_op]
            zeroes_and_ones_of_all_words.remove(word_list2)
            zeroes_and_ones_of_all_words.remove(word_list1)
            bitwise_op = [w1 & w2 for (w1, w2) in zip(word_list1, bitwise_op)]
            
            print("bitwise")
            print(bitwise_op)

    zeroes_and_ones_of_all_words.insert(0, bitwise_op)

    files = []
    lis = zeroes_and_ones_of_all_words[0]
    print('\n Hasil Incidence matrix')
    print(lis)
    #print("test")
    #print(zeroes_and_ones_of_all_words[0])
    #print("test2")
    #print(zeroes_and_ones_of_all_words[1])
    cnt = 1
    for index in lis:
        if index == 1:
            files.append('doc '+files_with_index[cnt])
        #else : (files.append('Kosong'))
        cnt = cnt + 1

    #Documen yang mengandung query
    print("\n Daftar Doc : \n"+str(files))
texts = [
        'New home sales top forecasts',
        'Home sales rise in july',
        'Increase in home sales in july',
        'July new home sales rise'
    ]
main(texts)


['home', 'and', 'july', 'not', 'sales']
['home', 'july', 'sales']
['and', 'not']
[[1, 1, 1, 1], [0, 1, 1, 1], [1, 1, 1, 1]]

 and arg
[[1, 1, 1, 1], [0, 1, 1, 1], [1, 1, 1, 1]]
[[0, 1, 1, 1], [1, 1, 1, 1]]

 not arg
[[0, 1, 1, 1], [1, 1, 1, 1]]
bitwise
[0, 0, 0, 0]

 Hasil Incidence matrix
[0, 0, 0, 0]

 Daftar Doc : 
[]
